# Sleep Dep, FULL DB
It looks like I can just get rid of the majority of these uninformative cols at the right end of the set. I will only take the following:
- subject_number
- image_name
- stain_type
- magnification_factor
- ROI_mean_intensity_from_PIPSQUEAK_withbackgroundsubtraction
- cohort
- ZT
- treatment
- sex


In [50]:
import numpy as np
import pandas as pd

# load data
df = pd.read_csv('FullDB_ROIsPhenotypesValidated.csv')

# check cols
print(df.columns)

# let's take a look
df.head()

/var/folders/b2/3h2lpxx14kgb12pp_7pltxnc0000gn/T/ipykernel_34760/387767506.py:5: DtypeWarning: Columns (22,25,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('FullDB_ROIsPhenotypesValidated.csv')


Index(['subject_number', 'image_name', 'stain_type', 'cell_number',
       'area_minus_background_micronsquared',
       'ROI_mean_intensity_from_PIPSQUEAK_withbackgroundsubtraction',
       'mean_intensity_beforebackgroundsubtraction', 'ROI_intensity_stdev',
       'ROI_minimum_intensity', 'ROI_maximum_intensity',
       'ROI_intensity_normalizedtoZT0_percohort_perstain',
       'magnification_factor', 'xm', 'ym', 'image_background_intensity',
       'cohort', 'ZT', 'location_wrt_bregma', 'treatment', 'sex',
       'exact_sac_time', 'weight_at_sac', 'analysisdate',
       'magnification_adjusted_area', 'which_stain', 'phenotypeGlobal',
       'phenotypeOxo', 'phenotypePV', 'phenotypeWFA', 'Is_Duplicate',
       'redundant_duplicate', 'confirmed_single', 'confirmed_double',
       'confirmed_triple', 'confirmed_AMBIGUOUS', 'unique_Cell_id',
       'empiricalPhenotype', 'Proximal_Coordinates', 'Valid_StainType',
       'sumConfirmed', 'real_stain_type', 'phenotypeWfa', 'valid_phenotype'

,subject_number,image_name,stain_type,cell_number,area_minus_background_micronsquared,ROI_mean_intensity_from_PIPSQUEAK_withbackgroundsubtraction,mean_intensity_beforebackgroundsubtraction,ROI_intensity_stdev,ROI_minimum_intensity,ROI_maximum_intensity,...,confirmed_triple,confirmed_AMBIGUOUS,unique_Cell_id,empiricalPhenotype,Proximal_Coordinates,Valid_StainType,sumConfirmed,real_stain_type,phenotypeWfa,valid_phenotype
0,RF056,RF056_5.16_L,Double-labeled_8-oxo_co-occurring_with_WFA,16,0.0,NaN,NaN,0.0,65535.0,NaN,...,N,N,0,NaN,NaN,N,0,NaN,NaN,NaN
1,RF056,RF056_2.76_R,8-oxo-dG,76,0.0,NaN,NaN,0.0,65535.0,NaN,...,N,N,0,NaN,NaN,N,0,NaN,NaN,NaN
2,RF056,RF056_5.16_L,8-oxo-dG,77,0.0,NaN,NaN,0.0,65535.0,NaN,...,N,N,0,NaN,NaN,N,0,NaN,NaN,NaN
3,RF056,RF056_5.16_L,Double-labeled_8-oxo_co-occurring_with_WFA,79,0.0,NaN,NaN,0.0,65535.0,NaN,...,N,N,0,NaN,NaN,N,0,NaN,NaN,NaN
4,RF056,RF056_2.76_R,8-oxo-dG,134,0.0,NaN,NaN,0.0,65535.0,NaN,...,N,N,0,NaN,NaN,N,0,NaN,NaN,NaN


In [55]:
# pull out cols that we need
df = df[['subject_number', 'image_name', 'stain_type', 'ROI_mean_intensity_from_PIPSQUEAK_withbackgroundsubtraction', 'cohort', 'ZT', 'treatment', 'sex', 'magnification_factor']]

# check for nans
df.count() - len(df)

# it looks like the mean_intensity col is missing 249 values. let's just remove those rows for now.
df = df[~df.ROI_mean_intensity_from_PIPSQUEAK_withbackgroundsubtraction.isna()]
df.head()

,subject_number,image_name,stain_type,ROI_mean_intensity_from_PIPSQUEAK_withbackgroundsubtraction,cohort,ZT,treatment,sex,magnification_factor
137,RF001,RF001_2.52_L,8-oxo-dG,15.4825,3.1,18,SD,F,1.633
138,RF001,RF001_2.52_L,Triple-labeled_8-oxo-dG,48.1643,3.1,18,SD,F,1.633
139,RF001,RF001_2.52_L,Double-labeled_8-oxo_co-occurring_with_WFA,48.1643,3.1,18,SD,F,1.633
140,RF001,RF001_2.52_L,8-oxo-dG,48.1643,3.1,18,SD,F,1.633
141,RF001,RF001_2.52_L,Double-labeled_8-oxo_co-occurring_with_WFA,29.5644,3.1,18,SD,F,1.633


In [82]:
def count_img_ns(name_col, df):
    '''
    takes a dataframe and counts the number of unique strings that occur in the 
    "image_name" col for each rat in "rat_n" col
    args:
        name_col: str, denoting the name of the col containing unique rat_n IDs
        df: pd.core.frame.DataFrame(n, m) 
            n: the number of rows, 
            m: the number of features
    return:
        df_imgn: pd.core.frame.DataFrame(n=|name_col|), m=2)
            n: the number of rows, equal to the cardinality of the name_col set
            (the number of unique ID strings in name_col)
            this df contains 2 cols: a name_col, and an image_n col
    '''
    assert 'image_name' in df.columns

    df_imgn = df.groupby([name_col])\
        .apply(lambda x: len(np.unique(x.image_name)))\
        .reset_index(name='image_n')
    
    return df_imgn

def count_cells_per_img(df):
    '''

    '''
    df_counts = df.value_counts(name_col, '')
    return df.value_counts(['rat_n', 'filename', 'treatment']).reset_index(name='cell_counts').sort_values(by=['rat_n', 'filename'])

SyntaxError: invalid syntax (525965362.py, line 24)

In [72]:
for stain in df.stain_type.unique():
    df_stain = df[df.stain_type == stain]
    

8-oxo-dG 			 507458
Triple-labeled_8-oxo-dG 			 16115
Double-labeled_8-oxo_co-occurring_with_WFA 			 57170
Double-labeled_8-oxo_co-occurring_with_PV 			 28555
Parvalbumin 			 32685
Double-labeled_PV_co-occurring_with_8-oxo 			 28529
Triple-labeled_Parvalbumin 			 16302
Double-labeled_PV_co-occurring_with_WFA 			 18608
Double-labeled_WFA_co-occurring_with_8-oxo 			 56682
WFA 			 85643
Triple-labeled_WFA 			 16335
Double-labeled_WFA_co-occurring_with_PV 			 18563


In [73]:
np.unique(df.image_name, return_counts=True)

(array(['RF001_2.52_L', 'RF001_2.52_R', 'RF001_3.0_L', 'RF001_3.0_R',
        'RF001_4.2_L', 'RF001_4.2_R', 'RF002_2.76_L', 'RF002_2.76_R',
        'RF002_3.72_L', 'RF002_3.72_R', 'RF002_4.68_L', 'RF002_4.68_R',
        'RF003_3.0_L', 'RF003_3.0_R', 'RF003_3.72_L', 'RF003_3.72_R',
        'RF003_4.68_L', 'RF003_4.68_R', 'RF004_2.76_L', 'RF004_2.76_R',
        'RF004_3.72_L', 'RF004_3.72_R', 'RF004_4.68_L', 'RF004_4.68_R',
        'RF005_2.76_L', 'RF005_2.76_R', 'RF005_3.72_L', 'RF005_3.72_R',
        'RF005_4.68_L', 'RF005_4.68_R', 'RF006_3.00_L', 'RF006_3.00_R',
        'RF006_3.72_L', 'RF006_3.72_R', 'RF006_4.68_L', 'RF006_4.68_R',
        'RF007_2.80_L', 'RF007_2.80_R', 'RF007_3.30_L', 'RF007_3.30_R',
        'RF007_4.68_L', 'RF007_4.68_R', 'RF008_3.0_L', 'RF008_3.0_R',
        'RF008_3.24_L', 'RF008_3.72_R', 'RF008_4.68_L', 'RF008_4.68_R',
        'RF009_2.52_L', 'RF009_2.52_R', 'RF009_3.24_L', 'RF009_3.24_R',
        'RF009_4.68_L', 'RF009_4.68_R', 'RF010_3.0_L', 'RF010_3.0_R',
  

In [81]:
'image_name' in df.columns

True

In [63]:
type(df)

pandas.core.frame.DataFrame